In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ultralytics import YOLO
import os
import shutil
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import pandas as pd
import numpy as np
from PIL import Image



In [ ]:
#src = str(Path.cwd().resolve().parents[1]/ 'src')
#sys.path.append(src)
#from config.paths import ROOT, DATA, EXPERIMENTS, DOC_IMGS, CLEANED, PROCESSED, METADATA

In [1]:
# Définition des chemins principaux
ROOT = '/content/drive/MyDrive/dataHN'
EXPERIMENTS = '/content/drive/MyDrive/dataHN/experiments'
DATA = '/content/drive/MyDrive/dataHN/data'

In [11]:
pip install ultralytics

In [27]:
def resize_save_img(input_path, output_path, target_size=640):
    img = Image.open(input_path).convert('RGB')
    w, h = img.size
    if w > h:
        new_w = target_size
        new_h = int(h * target_size / w)
    else:
        new_h = target_size
        new_w = int(w * target_size / h)
    img_resized = img.resize((new_w, new_h), Image.BILINEAR)
    img_resized.save(output_path, quality=90)


In [15]:
# Préparation du dataset

yolo_dir = os.path.join(EXPERIMENTS, 'yolo_obj_cls')
cls_dataset_dir = os.path.join(yolo_dir, 'dataset')  # contient un sous-dossier par classe


#data_dir = os.path.join(yolo_dir, 'data') # en local
data_dir = '/content/yolo_cls/data'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# Création des dossiers train / val
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

In [8]:
# Liste  des classes = sous-dossiers dans cls_dataset_dir
classes = [d for d in os.listdir(cls_dataset_dir) if os.path.isdir(os.path.join(cls_dataset_dir, d))]

for class_name in classes:
    class_path = os.path.join(cls_dataset_dir, class_name)
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    train_img, val_img = train_test_split(images, test_size=0.2, random_state=42)

    train_subfolder = os.path.join(train_dir, class_name)
    val_subfolder = os.path.join(val_dir, class_name)
    os.makedirs(train_subfolder, exist_ok=True)
    os.makedirs(val_subfolder, exist_ok=True)

    # Copie et redimensionnement pour train
    for img in tqdm(train_img, desc=f'{class_name} -> train'):
        src_path = os.path.join(class_path, img)
        dst_path = os.path.join(train_subfolder, img)
        resize_save_img(src_path, dst_path, target_size=640)

    # Copie et redimensionnement pour val
    for img in tqdm(val_img, desc=f'{class_name} -> val'):
        src_path = os.path.join(class_path, img)
        dst_path = os.path.join(val_subfolder, img)
        resize_save_img(src_path, dst_path, target_size=640)


face -> val: 100%|██████████| 7/7 [00:07<00:00,  1.10s/it]


In [4]:
model_name = 'yolo11s-cls.pt'
epochs = 40
batch = 16
dropout = 0.1
imgsz = 640

In [20]:
num_training = 4
training_dir = os.path.join(yolo_dir, 'training')
project = os.path.join(training_dir, 'runs')
os.makedirs(training_dir,exist_ok = True)
os.makedirs(project,exist_ok = True)
name = f'run{num_training}_{model_name.split(".")[0]}'


In [6]:
model = YOLO(model_name)
results = model.train(
    data=data_dir,  # train/val dataset
    epochs=epochs,
    batch=batch,
    dropout=dropout,
    imgsz=imgsz,
    project=project,
    name=name
)

Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_cls/data, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run4_yolo11s-cls, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

train: Scanning /content/yolo_cls/data/train... 60 images, 0 corrupt: 100%|██████████| 60/60 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 17.7±6.2 MB/s, size: 94.3 KB)


val: Scanning /content/yolo_cls/data/val... 15 images, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/dataHN/experiments/yolo_obj_cls/training/runs/run4_yolo11s-cls
Starting training for 40 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      40/40      3.11G   0.003719         12        640: 100%|██████████| 4/4 [00:02<00:00,  1.89it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 18.46it/s]

                   all      0.933          1



40 epochs completed in 0.033 hours.
Optimizer stripped from /content/drive/MyDrive/dataHN/experiments/yolo_obj_cls/training/runs/run4_yolo11s-cls/weights/last.pt, 11.0MB
Optimizer stripped from /content/drive/MyDrive/dataHN/experiments/yolo_obj_cls/training/runs/run4_yolo11s-cls/weights/best.pt, 11.0MB

Validating /content/drive/MyDrive/dataHN/experiments/yolo_obj_cls/training/runs/run4_yolo11s-cls/weights/best.pt...
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s-cls summary (fused): 47 layers, 5,436,690 parameters, 0 gradients, 12.0 GFLOPs
train: /content/yolo_cls/data/train... found 60 images in 2 classes ✅ 
val: /content/yolo_cls/data/val... found 15 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 14.84it/s]


                   all          1          1
Speed: 0.5ms preprocess, 3.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /content/drive/MyDrive/dataHN/experiments/yolo_obj_cls/training/runs/run4_yolo11s-cls


# Inference

In [ ]:
encheres_folder = os.path.join(DATA, 'raw_collected', 'objets', 'encheres') 
musees_folder = os.path.join(DATA, 'raw_collected', 'objets', 'musees') 
colab_data_dir = '/content/yolo/inference/obj_imgs'

print(len(os.listdir(encheres_folder)))
print(len(os.listdir(musees_folder)))

2687
272


In [24]:
shutil.copytree(encheres_folder, colab_data_dir, dirs_exist_ok=True)
shutil.copytree(musees_folder, colab_data_dir, dirs_exist_ok=True)

'/content/yolo/inference/obj_imgs'

In [21]:
model_best_path = os.path.join(project, name, 'weights','best.pt')


In [ ]:
predict_data = colab_data_dir
predict_images = [os.path.join(predict_data, img) for img in os.listdir(predict_data) if img.endswith('.jpg')]
len(predict_images)

2959

In [23]:
predicts_dir = os.path.join(yolo_dir, 'predicts')
project = os.path.join(predicts_dir, 'runs')
name = f'run{num_training}_{model_name.split(".")[0]}'
csv_dir = os.path.join(predicts_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)
results_csv = os.path.join(csv_dir, f'{name}_prediction_results.csv')

In [24]:


# Split pour éviter crash RAM sur colab
sections = np.array_split(predict_images, 20)
sections = [list(section) for section in sections]

model_best = YOLO(model_best_path)


for i, section_data in enumerate(sections):
    print(f"Processing section {i+1}/20...")

    results = model_best.predict(section_data, project=project, name = name, device = 'cuda:0', batch=2)

    all_results = []
    for result in results:
        probs = result.probs
        top1_idx = probs.top1
        top1_conf = probs.top1conf.item()
        class_names = result.names

        all_results.append({
            'image': result.path,
            'predicted_class': class_names[top1_idx],
            'confidence': top1_conf
        })

    df_section = pd.DataFrame(all_results)
    df_section.to_csv(results_csv, mode='a', index=False, header=(i == 0))


Processing section 1/20...

0: 640x640 face 0.93, revers 0.07, 5.0ms
1: 640x640 face 1.00, revers 0.00, 5.0ms
2: 640x640 revers 0.70, face 0.30, 5.0ms
3: 640x640 face 1.00, revers 0.00, 5.0ms
4: 640x640 revers 1.00, face 0.00, 5.0ms
5: 640x640 face 1.00, revers 0.00, 5.0ms
6: 640x640 face 0.99, revers 0.01, 5.0ms
7: 640x640 revers 0.73, face 0.27, 5.0ms
8: 640x640 face 0.97, revers 0.03, 5.0ms
9: 640x640 face 0.71, revers 0.29, 5.0ms
10: 640x640 face 0.99, revers 0.01, 5.0ms
11: 640x640 face 0.96, revers 0.04, 5.0ms
12: 640x640 face 0.97, revers 0.03, 5.0ms
13: 640x640 face 0.97, revers 0.03, 5.0ms
14: 640x640 revers 0.78, face 0.22, 5.0ms
15: 640x640 face 0.99, revers 0.01, 5.0ms
16: 640x640 revers 1.00, face 0.00, 5.0ms
17: 640x640 face 0.97, revers 0.03, 5.0ms
18: 640x640 revers 0.61, face 0.39, 5.0ms
19: 640x640 revers 0.52, face 0.48, 5.0ms
20: 640x640 face 0.99, revers 0.01, 5.0ms
21: 640x640 face 1.00, revers 0.00, 5.0ms
22: 640x640 revers 1.00, face 0.00, 5.0ms
23: 640x640 reve

In [25]:
all_results[1]

{'image': np.str_('/content/yolo/inference/obj_imgs/GazetteDrouot_15253844_01.jpg'),
 'predicted_class': 'face',
 'confidence': 0.9529070258140564}

In [ ]:

csv_path = results_csv  # chemin vers le fichier CSV
output_dir = os.path.join(predicts_dir, "sorted_results")
resize_target = 256  # taille cible pour verification

face_dir = os.path.join(output_dir, "face")
revers_dir = os.path.join(output_dir, "revers")
os.makedirs(face_dir, exist_ok=True)
os.makedirs(revers_dir, exist_ok=True)

df = pd.read_csv(csv_path)

for _, row in df.iterrows():
    img_path = row["image"]
    predicted_class = row["predicted_class"]

    if not os.path.isfile(img_path):
        print(f"Image introuvable : {img_path}")
        continue

    try:
        filename = os.path.basename(img_path)
        if predicted_class == "face":
            save_path = os.path.join(face_dir, filename)
        else:
            save_path = os.path.join(revers_dir, filename)

        resize_save_img(img_path, save_path, target_size=resize_target)

    except Exception as e:
        print(f"Erreur avec {img_path} : {e}")

In [29]:
print('face :', len(os.listdir(face_dir)))
print('revers:', len(os.listdir(revers_dir)))

face : 2198
revers: 761
